#### Week 3 Assignment TextBlob
##### Michael Perrine
##### DSC 550 Data Mining
##### Professor Werner

# Part 1 Using the TextBlob Sentiment Analyzer

In [ ]:
import pandas as pd    
from textblob import TextBlob

1. Import the movie review data as a data frame and ensure that the data is loaded properly.

In [1]:
# This code loads the text data for the movie reviews
text = pd.read_table(r"C:\Users\mchlp\OneDrive\Documents\Data Mining\labeledTrainData.tsv", encoding = 'latin-1')

NameError: name 'pd' is not defined

In [ ]:
# This code checks the first 5 rows to verify it loaded correctly
text.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


2. How many of each positive and negative reviews are there?

In [ ]:
# This code creates a dataframe of just the sentiment of the movies
sent = text['sentiment']

In [ ]:
# This code shows the dimension of the dataframe
sent.shape

(25000,)

In [ ]:
# This function counts the number of positive and negative sentiment scores

def count_positive_negative_reviews(sent):
    
    positive = 0
    negative = 0
    for i in sent:
        if i == 1: 
            positive += 1
        if i == 0:
            negative += 1
    return(positive, negative)
count_positive_negative_reviews(sent)

(12500, 12500)

3. Use TextBlob to classify each movie review as positive or negative. Assume that a polarity score greater than or equal to zero is a positive sentiment and less than 0 is a negative sentiment.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# This code creates a textblob object that includes the polarity and subjectivity 

new_text = []


for r in text['review']:
    blob = TextBlob(str(r))
    text_1 = {
        'polarity':blob.sentiment.polarity,
        'subjectivity':blob.sentiment.subjectivity
    }
    new_text.append(text_1)


In [ ]:
# This code creates a new data frame out of the text data
new_text_df = pd.DataFrame(new_text)

In [ ]:
# This code validates that the data frame was created
new_text_df.head()

,polarity,subjectivity
0,0.001277,0.606746
1,0.256349,0.531111
2,-0.053941,0.562933
3,0.134753,0.492901
4,-0.024842,0.459818


In [ ]:
# This code adds polarity to the text data frame
text.insert(3,'review_polarity', new_text_df['polarity'])

In [ ]:
# This code adds subjectivity to the text data frame
text.insert(4, 'review_subjectivity', new_text_df['subjectivity'])

In [ ]:
# This code validates the new columns in the text data frame
text.head()

,id,sentiment,review,review_polarity,review_subjectivity
0,5814_8,1,With all this stuff going down at the moment w...,0.001277,0.606746
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",0.256349,0.531111
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,-0.053941,0.562933
3,3630_4,0,It must be assumed that those who praised this...,0.134753,0.492901
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,-0.024842,0.459818


In [ ]:
# This code creates a new data frame adding review_polarity to the pol data frame for the 
# purpose of counting the positive and negative sentiment scores.

pol = text['review_polarity']

In [ ]:
# This code creates a function to separate the positive and negative reviews

def count_positive_negative_reviews(pol):
    
    positive = 0
    negative = 0
    for i in pol:
        if i >= 0:
            positive += 1
        if i <= 0:
            negative += 1
    return(positive, negative)
count_positive_negative_reviews(pol)

(19017, 6000)

4. Check the accuracy of this model. Is this model better than random guessing?

# Part 2 Preparing Text for a Custom Model

1. Convert all text to lowercase letters.


In [ ]:
lower = text['review']


def lower_case(string:str)-> str:
    return string.lower()

[lower_case(string) for string in lower]

2. Remove punctuation and special characters from the text.


In [ ]:
# Import libraries
import unicodedata
import sys

In [ ]:
# Create dictionary to remove punctuations
punctuation = dict.fromkeys((i for i in range(sys.maxunicode)\
    if unicodedata.category(chr(i)).startswith('P')), None)

In [ ]:
# Apply code to remove punctuation
lower_1 = [string.translate(punctuation) for string in lower]
lower_1

3. Remove stop words.


In [111]:
# load libraries
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mchlp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Import stop words dictionary
stop_words = stopwords.words('english')

In [ ]:
# This code removes words in stop words list
[word for word in lower_1 if word not in stop_words]

 4. Apply NLTK’s PorterStemmer.


In [115]:
# Load libraries
from nltk.stem.porter import PorterStemmer

In [116]:
porter = PorterStemmer()

In [ ]:
lower_2 = [porter.stem(word) for word in lower_1]
lower_2

5. Create a bag-of-words matrix from your stemmed text (output from (4)), where each row is a word-count vector for a single movie review. Display the dimensions of your bag-of-words matrix. The number of rows in this matrix should be the same as the number of rows in your original data frame.
.

In [120]:
# Load Library 
import numpy as np  
from sklearn.feature_extraction.text import CountVectorizer     

In [123]:
# Create the bag of words matrix
count= CountVectorizer()
bag_of_words = count.fit_transform(lower_2)

In [124]:
bag_of_words

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 3453417 stored elements and shape (25000, 112640)>

In [125]:
bag_of_words.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(25000, 112640))

6. Create a term frequency-inverse document frequency (tf-idf) matrix from your stemmed text, for your movie reviews. Display the dimensions of your tf-idf matrix. These dimensions should be the same as your bag-of-words matrix

In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [127]:
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(lower_2)

In [128]:
feature_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3453417 stored elements and shape (25000, 112640)>

In [129]:
feature_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(25000, 112640))